In [ ]:
import os
path_to_frontend = '..'
os.chdir(path_to_frontend)
import q100viz.session as session
import q100viz.devtools as devtools
import q100viz.graphics.graphs as graphs

In [ ]:
devtools.select_buildings_for_simulation(session.buildings.df, ['2.19', '5.14', '7.09'])

In [ ]:
session.simulation.activate(max_year=2045)